In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [22]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("hello, how, are, you")

['hello', 'how', 'are', 'you']

In [23]:
template = ChatPromptTemplate.from_messages(
  [
    ('system', 'You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lower case. Do NOT replay with anything else.'),
    ('human', "{question}") 
  ]
)

prompt = template.format_messages(
  max_items=10,
  question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'black',
 'white',
 'pink',
 'brown']

In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke({
  'max_items': 10,
  'question': "What are the pokemons?"
})